In [1]:
import geopandas as gpd
from shapely.geometry import Point

# Load the dataset
from geopandas.datasets import get_path

cities = gpd.read_file(get_path('naturalearth_cities'))


/tmp/ipykernel_17374/464501985.py:7: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_cities' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  cities = gpd.read_file(get_path('naturalearth_cities'))


In [2]:
import pandas as pd

In [3]:
countries = pd.read_csv("data/country_info.csv")
countries

,name,code,capital
0,Albania,al,Tirana
1,Armenia,am,Yerevan
2,Australia,au,Canberra
3,Austria,at,Vienna
4,Azerbaijan,az,Baku
5,Belgium,be,Brussels
6,Croatia,hr,Zagreb
7,Cyprus,cy,Nicosia
8,Czechia,cz,Prague
9,Denmark,dk,København


In [4]:
# Filter for European capitals (manually or with an additional dataset)
european_capitals = countries["capital"]

european_cities = cities[cities['name'].isin(european_capitals)]

# Reproject to a metric CRS (EPSG:3035)
european_cities = european_cities.to_crs(epsg=3035)

# Compute pairwise distances
distances = european_cities.geometry.apply(
    lambda city: european_cities.distance(city)
)

# Convert distances to a DataFrame for readability
import pandas as pd
distance_df = pd.DataFrame(distances.values.tolist(),
                           index=european_cities['name'],
                           columns=european_cities['name'])
distance_df_km = distance_df / 1000
distance_df_km.head()

name,San Marino,Bratislava,Bern,Reykjavík,Chi?in?u,Vilnius,Riga,Zagreb,Tallinn,Tbilisi,...,Kyiv,Madrid,Stockholm,Amsterdam,Berlin,Athens,Vienna,London,Rome,Paris
name,,,,,,,,,,,,,,,,,,,,,
San Marino,0.000000,592.189117,511.086762,3099.359842,1326.956807,1514.094599,1662.699421,349.805603,1914.409177,2625.451178,...,1544.418564,1387.303070,1753.433237,1088.861755,957.030918,1152.241692,563.123967,1260.407706,225.783903,948.745271
Bratislava,592.189117,0.000000,739.318867,2922.625840,889.218718,923.239099,1087.012101,274.895322,1350.372486,2261.394482,...,1004.274806,1860.351135,1245.520598,985.789892,553.540765,1243.935347,56.224988,1291.093344,784.856524,1092.144862
Bern,511.086762,739.318867,0.000000,2613.292005,1621.684829,1518.674917,1589.307121,668.443283,1795.844146,2983.262618,...,1732.612245,1148.982118,1546.997047,631.984668,755.572555,1655.955050,684.675850,751.759779,685.090392,439.418758
Reykjavík,3099.359842,2922.625840,2613.292005,0.000000,3579.890883,2798.602074,2582.512481,3080.708812,2459.901546,4810.805124,...,3378.262910,2903.247623,2139.417689,2013.258273,2382.413311,4150.844283,2883.413069,1886.004345,3295.703610,2229.303861
Chi?in?u,1326.956807,889.218718,1621.684829,3579.890883,0.000000,891.632377,1156.162476,996.374266,1414.603583,1375.386664,...,402.958789,2701.774139,1547.134580,1814.238083,1264.931856,1091.873708,945.391764,2149.167707,1421.695209,1978.495135


In [5]:
distance_df_km.describe()

name,San Marino,Bratislava,Bern,Reykjavík,Chi?in?u,Vilnius,Riga,Zagreb,Tallinn,Tbilisi,...,Kyiv,Madrid,Stockholm,Amsterdam,Berlin,Athens,Vienna,London,Rome,Paris
count,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,...,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000
mean,1666.731846,1493.586783,1700.543717,3291.086028,1729.467547,1697.705100,1770.929060,1547.220355,1927.893478,2608.044654,...,1770.282489,2502.613623,1866.062544,1762.666181,1558.997452,2010.982789,1494.424378,1964.882596,1772.265992,1861.549755
std,1944.823369,1887.327298,2004.184975,2034.231825,1718.215960,1772.617626,1794.784876,1902.169295,1787.076310,1595.852885,...,1692.115678,2113.992688,1856.353003,2026.720441,1924.957727,1790.149309,1897.968194,2078.519882,1939.258969,2063.884321
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,904.411281,784.856524,739.318867,2382.413311,996.374266,891.632377,847.021272,771.012290,1071.626613,1980.145486,...,1071.626613,1477.798222,1053.894231,916.141838,771.012290,1152.241692,764.104338,1157.103711,921.582379,1028.259319
50%,1305.658896,1087.012101,1351.969654,2903.247623,1421.695209,1472.312868,1458.470248,1125.034383,1675.777051,2456.172090,...,1491.580339,2070.942588,1546.997047,1371.664618,1040.353793,1816.268404,1055.048057,1578.485825,1421.695209,1448.443265
75%,1889.128947,1737.035689,1732.612245,3662.987302,1814.238083,2056.138993,2233.834856,1705.401383,2402.607575,3065.049623,...,1840.105001,2864.578659,2405.404678,1855.412523,1843.826130,2382.618978,1685.093285,2092.191809,1975.409446,1978.495135
max,12263.769177,11801.774379,12539.566033,13413.100915,10942.552079,11162.566740,11237.550563,11938.925902,11206.257690,9678.622477,...,10811.680077,13644.326045,11584.829782,12533.521711,11969.525694,11503.216179,11855.583155,12891.318423,12325.732706,12830.105202


In [6]:
len(european_capitals), len(european_cities)

(38, 37)

In [7]:
set(european_capitals) - set(european_cities["name"])

{nan}